In [ ]:
import sys
import asyncio
import time
import os
import json

import numpy as np

from lsst.ts import salobj

# Import the control classesf
# the MTCS might not be needed yet so I'll comment it out
# from lsst.ts.observatory.control.maintel.mtcs import MTCS
from lsst.ts.observatory.control.maintel.comcam import ComCam

In [ ]:
# for tab completion to work
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
# Just verify that the right environment variables are loaded
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# grab the communication domain
domain = salobj.Domain()

In [ ]:
#mtcs = MTCS(domain)
comcam = ComCam(domain)
# Create Remote to ocps 
ocps = salobj.Remote(domain, "OCPS")
await asyncio.gather(comcam.start_task, ocps.start_task) #, mtcs.start_task)

### When writing a script, the components should (probably) be enabled by a user.

In [ ]:
await comcam.rem.cccamera.evt_heartbeat.next(flush=True, timeout=10)

In [ ]:
# Bring ComCam (and other components to the enabled state)
await comcam.enable()

In [ ]:
# now enable the OCPS, and supply a configuration
await salobj.set_summary_state(ocps, salobj.State.ENABLED, settingsToApply="LSSTComCam.yaml")

In [ ]:
?comcam.take_bias

In [ ]:
# take n biases, returns a list of the obsids
tmp = await comcam.take_bias(2)
print(tmp)

In [ ]:
# did the images get archived and are they available to the butler?
val = await comcam.rem.ccarchiver.evt_imageInOODS.aget(timeout=10)
print(val)

In [ ]:
# Question: How do we verify that each file got ingested and not just check the latest?

In [ ]:
# Should verify that the OODS ingestion is equal to the last obsid in the set of biases

In [ ]:
# now send to the OCPS
exposures = tuple(tmp)
detectors = (0,1,2,3,4,5,6,7,8)
# can we get instrument out of anything in the control system classes?
instrument='LSSTComCam' # must be LATISS, LSSTComCam or LSSTCam

In [ ]:
ack = await ocps.cmd_execute.set_start(
    wait_done=False,
    pipeline="${CP_PIPE_DIR}/pipelines/cpBias.yaml", version="",
    config="-j 9 -c isr:doDefect=False",
    data_query=f"instrument='{instrument}' AND"
               f" detector IN {detectors} AND exposure IN {exposures}"
)
if ack.ack != salobj.SalRetCode.CMD_ACK:
    ack.print_vars()

In [ ]:
# Wait for the in-progress acknowledgement with the job identifier.  This can be executed immediately after the previous cell.
ack = await ocps.cmd_execute.next_ackcmd(ack, wait_done=False, timeout=10)
print('Received acknowledgement of ocps command')
ack.print_vars()
job_id = json.loads(ack.result)["job_id"]

# Wait for the command completion acknowledgement.  This can be executed anytime after the execute command.
ack = await ocps.cmd_execute.next_ackcmd(ack, timeout=600)
print('Received command completion acknowledgement from  ocps')
if ack.ack != salobj.SalRetCode.CMD_COMPLETE:
    ack.print_vars()
    
#Wait for the job result message that matches the job id we're interested in, ignoring any others (from other remotes).
# This obviously needs to follow the first acknowledgement (that returns the job id) but might as well wait for the second.
while True:
    msg = await ocps.evt_job_result.next(flush=False, timeout=10)
    response = json.loads(msg.result)
    if response["jobId"] == job_id:
        break
print(response)

In [ ]:
# can run independently if it times out.
msg = await ocps.evt_job_result.next(flush=False, timeout=10)

In [ ]:
print(msg)

In [ ]:
# Put instrument back in standby
await comcam.standby()

In [ ]:
# Put OCPS back in standby
await salobj.set_summary_state(ocps, salobj.State.STANDBY)